In [ ]:
#!pip install -q torch==1.7.1+cu101 torchvision==0.8.2+cu101 torchaudio==0.7.2 torchtext==0.8.1 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
!nvidia-smi

In [ ]:
#!pip install -q --no-index torch-scatter -f https://pytorch-geometric.com/whl/torch-1.7.0+cu101.html
#!pip install -q --no-index torch-sparse -f https://pytorch-geometric.com/whl/torch-1.7.0+cu101.html
#!pip install -q --no-index torch-cluster -f https://pytorch-geometric.com/whl/torch-1.7.0+cu101.html
#!pip install -q --no-index torch-spline-conv -f https://pytorch-geometric.com/whl/torch-1.7.0+cu101.html
#!pip install torch-geometric
#!pip install optuna

In [ ]:
import torch
import optuna
import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from sklearn.neural_network import MLPClassifier 
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from torch_geometric.data import NeighborSampler
from torch.optim import lr_scheduler
import torch.optim as optim
import matplotlib.pyplot as plt
import collections
import numpy as np
from modules.model import Net
from sklearn.metrics import f1_score
from modules.sampling import Sampler, SamplerContextMatrix, SamplerRandomWalk,SamplerFactorization,SamplerAPP
from datetime import datetime
import random
from torch_geometric.data import GraphSAINTNodeSampler

In [ ]:
dataset =Planetoid(root='/tmp/Cora', name='Cora',transform=T.NormalizeFeatures())
data = dataset[0]
indices_sh=list(range(len(data.x)))
indices=list(range(len(data.x)))
random.shuffle(indices_sh)
data.x
data.y
data.edge_index
x_new_map={}
for i,x in enumerate(indices_sh):
    x_new_map[x] = i
data_x_new = data.x[indices_sh]
data_y_new = data.x[indices_sh]
data_edge_index = torch.tensor(list(map(lambda x: [x_new_map[int(x[0])],x_new_map[int(x[1])]] ,data.edge_index.t()))).t()

data.x=data_x_new
data.edge_index=data_edge_index
data.y=data_y_new
    
train_indices = torch.tensor(indices[:int(0.7*len(indices)+1)])
val_indices = torch.tensor(indices[int(0.7*len(indices)+1):int(0.8*len(indices)+1)])
test_indices = torch.tensor(indices[int(0.8*len(indices)+1):])
train_mask = torch.tensor([False]*len(indices))
test_mask = torch.tensor([False]*len(indices))
val_mask = torch.tensor([False]*len(indices))
train_mask[train_indices] =True
test_mask[test_indices]=True
val_mask[val_indices]=True
        

In [ ]:
import pickle
import os

class Main():
    def __init__(self,conv, device, loss_function, mode ,dataset,data,train_indices,val_indices,test_indices,train_mask,val_mask,test_mask):
        self.dataset =Planetoid(root='/tmp/Cora', name='Cora',transform=T.NormalizeFeatures())
        data = self.dataset[0]
        self.Conv = conv
        self.device = device
        self.dataset = dataset
        self.x = data.x
        self.y = data.y.squeeze()
        self.data=data.to(device)
        self.loss = loss_function
        self.mode = mode
        self.train_indices =train_indices# torch.tensor(indices[:int(0.7*len(indices)+1)])
        self.val_indices =val_indices# torch.tensor(indices[int(0.7*len(indices)+1):int(0.8*len(indices)+1)])
        self.test_indices = test_indices#torch.tensor(indices[int(0.8*len(indices)+1):])
        self.train_mask = train_mask#torch.tensor([False]*len(indices))
        self.test_mask = test_mask#torch.tensor([False]*len(indices))
        self.val_mask =val_mask# torch.tensor([False]*len(indices))
        self.flag = self.loss["flag_tosave"]
        
        super(Main, self).__init__()
    def sampling(self,Sampler,epoch,nodes):
        if (epoch == 0): 
            if self.flag:  
                if "alpha" in self.loss: 
                    name_of_file = "samples_"+self.loss["Name"]+"_alpha_"+str(self.loss["alpha"])+".pickle"
                else:
                    name_of_file = "samples_"+self.loss["Name"]+".pickle"
                
                if os.path.exists(name_of_file):
                    with open(name_of_file,'rb') as f:
                        self.samples = pickle.load(f)
                else:
                    self.samples = Sampler.sample(nodes) 
                    with open(name_of_file,'wb') as f:
                        pickle.dump(self.samples,f)
            else:
                
                self.samples = Sampler.sample(nodes)
 
    def train(self, model,data,optimizer,Sampler,train_loader,dropout,epoch):
        model.train()   
        total_loss = 0
        optimizer.zero_grad()
        if model.mode == 'unsupervised':
            if model.conv=='GCN':
                arr=torch.nonzero(self.train_mask == True)
                indices_of_train_data = ([item for sublist in arr for item in sublist])
                out = model.inference(data.to(device),dp=dropout)
                samples = self.sampling(Sampler,epoch, indices_of_train_data)
                loss = model.loss(out[self.train_mask], self.samples)
                total_loss+=loss
            else:
                for batch_size, n_id, adjs in train_loader:
                    if len(train_loader.sizes) == 1:
                        adjs = [adjs]
                    adjs = [adj.to(device) for adj in adjs]
                    out = model.forward(data.x[n_id.to(device)].to(device), adjs)
                    self.sampling(Sampler,epoch,n_id[:batch_size])                 
                    loss = model.loss(out, self.samples)#pos_batch.to(device), neg_batch.to(device))
                    total_loss+=loss
            total_loss.backward()
            optimizer.step()      
            return total_loss /len(train_loader)
        elif model.mode== 'supervised':
            if model.conv=='GCN':
                out = model.inference(data.to(device),dp=dropout)
                loss = model.loss_sup(out[self.train_mask],y[self.train_mask])
                total_loss+=loss
            else:
                for batch_size, n_id, adjs in train_loader:
                    adjs = [adj for adj in adjs]
                    out = model.forward(data.x[n_id].to(device), adjs)
                    loss = model.loss_sup(out,y[n_id[:batch_size]])
                    total_loss+=loss
            total_loss.backward(retain_graph=True)
            optimizer.step()      
            return total_loss /len(train_loader)       

    @torch.no_grad()
    def test(self, model,data,classifier,**kwargs):#,n_estimators,learning_rate_carboost, max_depth): 
        model.eval()
        out = model.inference(data.to(device))
        y_true = self.y.detach().numpy()
        if model.mode == 'supervised':
            y_true = self.y.unsqueeze(-1)
            y_pred = out.cpu().argmax(dim=-1, keepdim=True)
            accs = []
            for mask in [self.train_mask, self.val_mask, self.test_mask]:    
                accs+=[int(y_pred[mask].eq(y_true[mask]).sum()) / int(mask.sum())]
            return accs
        elif model.mode == 'unsupervised': 
            if classifier == 'logistic regression':
                clf = LogisticRegression(max_iter = 3000,warm_start=True).fit(out.cpu().detach()[self.train_mask].numpy(), self.y.detach()[self.train_mask].numpy())
            else:
                n_estimators= kwargs["n_estimators"]
                learning_rate_catboost = kwargs["learning_rate_catboost"]
                max_depth = kwargs["max_depth"]
                clf = GradientBoostingClassifier(n_estimators=n_estimators, learning_rate = learning_rate_catboost, max_depth=max_depth, random_state=0)
                clf.fit(out.cpu().detach()[self.train_mask].numpy(), self.y.detach()[self.train_mask].numpy())
            accs = []
            for mask in [self.train_mask,self.test_mask,self.val_mask]:
                accs += [f1_score(self.y.detach()[mask].cpu().numpy(),clf.predict(out.cpu().detach()[mask].numpy()), average='macro')]
                
            return accs

    def run(self,hidden_layer=64,out_layer=128,dropout=0.0,size=1,learning_rate=0.001):
        

        classifier = "logistic regression"
        train_loader = NeighborSampler(self.data.edge_index, node_idx=self.train_mask, batch_size = int(sum(self.train_mask)), sizes=[-1]*size)
        Sampler = self.loss["Sampler"]
        LossSampler = Sampler(self.data,device=device,mask=self.train_mask,loss_info=self.loss)
        model = Net(dataset = self.dataset,mode=self.mode,conv=self.Conv,loss_function=self.loss,device=device,hidden_layer=hidden_layer,out_layer =out_layer,num_layers = (size),dropout = dropout)
        model.to(device)

        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay = 1e-5)
                #scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10, threshold=0.01, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08, verbose=False)
        scheduler=lr_scheduler.StepLR(optimizer, step_size=25,gamma=0.1)
        losses=[]
        train_accs=[]
        test_accs=[]
        val_accs=[]
        name_of_plot='conv: '+model.conv+', mode: '+model.mode+', loss from '+self.loss["Name"]

        print(name_of_plot)

        for epoch in range(100):
                    print('epoch',epoch)
                    loss = self.train(model,self.data,optimizer,LossSampler,train_loader,dropout,epoch)
                    losses.append(loss)
                    train_acc, test_acc,val_acc = self.test(model,self.data,'logistic regression')
                    train_accs.append(train_acc)
                    test_accs.append(test_acc)
                    log = 'Loss: {:.4f}, Epoch: {:03d}, Train: {:.4f}, Test: {:.4f}'
                    #scheduler.step()
                    print(log.format(loss, epoch, train_acc, test_acc))
        print('Test acc on the last epoch ', test_acc)
        plt.plot(losses)
        plt.title(name_of_plot+' loss')
        plt.xlabel('epoch')
        plt.ylabel('loss')
        plt.show()
        plt.plot(test_accs)
        plt.title(name_of_plot+' loss')
        plt.xlabel('epoch')
        plt.ylabel('loss')
        plt.show()

In [ ]:
class MainOptuna(Main):
    def objective(self,trial):
        # Integer parameter
        hidden_layer = trial.suggest_categorical("hidden_layer", [32,64,128,256])
        out_layer = trial.suggest_categorical("out_layer", [32,64,128])
        dropout = trial.suggest_float("dropout", 0.0,0.5,step = 0.1)
        size = trial.suggest_categorical("size of network, number of convs", [1,2,3])
        Conv = self.Conv
        learning_rate= trial.suggest_float("lr",5e-4,1e-2)

        # варьируем параметры
        loss_to_train={}
        for name in self.loss:
            
            if type(self.loss[name]) == list :
                if len(self.loss[name]) == 3:
                    var = trial.suggest_int(name,self.loss[name][0],self.loss[name][1],step=self.loss[name][2])
                    loss_to_train[name] = var
                elif len(self.loss[name]) == 2:
                    var_2 = trial.suggest_float(name,self.loss[name][0],self.loss[name][1])
                    loss_to_train[name] = var_2
                else:
                    var_3 = trial.suggest_categorical(name, self.loss[name])
                    loss_to_train[name] = var_3
            else:
                loss_to_train[name] = self.loss[name]
        Sampler =loss_to_train["Sampler"]
        model = Net(dataset = self.dataset,mode='unsupervised',conv=Conv,loss_function=loss_to_train,device=device,hidden_layer=hidden_layer,out_layer =out_layer,num_layers = size,dropout = dropout)
        train_loader = NeighborSampler(self.data.edge_index, batch_size = int(sum(self.train_mask)),node_idx=self.train_mask, sizes=[-1]*size)
       # train_loader = NeighborSampler(self.data.edge_index, batch_size = int(sum(self.train_mask)), sizes=[-1]*size)
        LossSampler = Sampler(self.data,device=self.device,mask=self.train_mask,loss_info=loss_to_train)
        model.to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay = 1e-5)  

        classifier = "logistic regression" #trial.suggest_categorical("classifier", ["logistic regression", "catboost"])

        if classifier == "catboost":
            n_estimators = trial.suggest_int("n of estimators", 10,40,5)
            learning_rate_catboost = trial.suggest_float("lr_catboost",5e-4,1e-2)
            max_depth = trial.suggest_int("max_depth",1,10,2)
        else:
            n_estimators = -1
            learning_rate_catboost =-1
            max_depth = -1
        #training of the model
        for epoch in range(50):
            loss = self.train(model,self.data,optimizer,LossSampler,train_loader,dropout,epoch)
            #self.test(model,self.data,classifier,n_estimators=n_estimators,learning_rate_catboost=learning_rate_catboost,max_depth=max_depth)
        train_acc, test_acc,val_acc = self.test(model,self.data,classifier,n_estimators=n_estimators,learning_rate_catboost=learning_rate_catboost,max_depth=max_depth)
        trial.report(val_acc,epoch)
        return val_acc

    
    def run(self,number_of_trials):

        study = optuna.create_study(direction="maximize",study_name=self.loss["Name"]+" loss,"+str(self.Conv)+" conv")
        study.optimize(self.objective,n_trials = number_of_trials)

        print('Best trial:')
        trial = study.best_trial
        print(" Value: ", trial.value)
        print(" Params: ")
        for key, value in trial.params.items():
            print(" {}: {}".format(key,value))

In [ ]:
from modules.sampling import Sampler, SamplerContextMatrix, SamplerRandomWalk,SamplerFactorization,SamplerAPP
#хорошо
DeepWalk = {"Name": "DeepWalk","walk_length":[5,20,5],"walks_per_node":[5,20,5],"num_negative_samples":[1,21,5],"context_size" : [5,20,5],"p":1,"q":1,"loss var": "Random Walks","flag_tosave":False,"Sampler" : SamplerRandomWalk } #Проблемы с памятью после того, как увеличила количество тренировочных данных
Node2Vec = {"Name": "Node2Vec","walk_length":[5,20,5],"walks_per_node":[5,20,5],"num_negative_samples":[1,21,5],"context_size" : [5,20,5],"p": [0.0,0.9] ,"q":[0.0,0.9], "loss var": "Random Walks","flag_tosave":False,"Sampler": SamplerRandomWalk}#то же самое 
LINE = {"Name": "LINE","C": "Adj","num_negative_samples":[1,21,5],"loss var": "Context Matrix","flag_tosave":True,"Sampler" :SamplerContextMatrix} 

#плохо
HOPE_RPR = {"Name": "HOPE_RPR","C":"RPR","loss var": "Factorization","flag_tosave":False,"alpha": [0,1],"Sampler" :SamplerFactorization} #проверить
HOPE_Katz = {"Name": "HOPE_Katz","C":"Katz","loss var": "Factorization","flag_tosave":False,"betta": [0,1],"Sampler" :SamplerFactorization,} #проверить
HOPE_AA = {"Name": "HOPE_AdamicAdar","C":"AA","loss var": "Factorization","flag_tosave":False,"Sampler" :SamplerFactorization} 
LapEigen = {"Name": "LaplacianEigenMaps", "C":"Adj","loss var": "Laplacian EigenMaps","flag_tosave":True,"Sampler" :SamplerFactorization}
VERSE_PPR =  {"Name": "VERSE_PPR","C": "PPR","num_negative_samples":[1,21,5],"loss var": "Context Matrix","flag_tosave":False,"alpha": [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9],"Sampler" :SamplerContextMatrix}

#под вопросом
VERSE_SR =  {"Name": "VERSE_SimRank","C": "SR","num_negative_samples":[1,21,5],"loss var": "Context Matrix","flag_tosave":True,"Sampler":SamplerContextMatrix} 

Struc2Vec ={} #Implement
#в приципе можно найти нормальны результат через оптюну, но в основном плохо
GraphFactorization = {"Name": "Graph Factorization","C":"Adj","loss var": "Factorization","flag_tosave":False,"Sampler" :SamplerFactorization} 
HOPE_CN = {"Name": "HOPE_CommonNeighbors","C":"CN","loss var": "Factorization","flag_tosave":False,"Sampler" :SamplerFactorization} 
APP ={"Name": "APP","C": "PPR","num_negative_samples":[1,21,5],"loss var": "Context Matrix","flag_tosave":False,"alpha": [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9],"Sampler" :SamplerAPP}
VERSE_Adj =  {"Name": "VERSE_Adj","C": "Adj","num_negative_samples":[1,21,5],"loss var": "Context Matrix","flag_tosave":False,"Sampler" :SamplerContextMatrix} 


In [ ]:
device =  torch.device('cuda',1)# if torch.cuda.is_available() else 'cpu')
loss = DeepWalk
MO = MainOptuna('SAGE', device, loss , 'unsupervised',dataset,data,train_indices,val_indices,test_indices,train_mask,val_mask,test_mask)
MO.run(number_of_trials=30)

In [ ]:
device =  torch.device('cuda' if torch.cuda.is_available() else 'cpu')

loss =  HOPE_CN
MO = Main('SAGE', device, loss , mode = 'unsupervised')
MO.run(hidden_layer=64,out_layer=64,dropout=0.5,size=1,learning_rate=0.007476328271512487)

In [ ]:
device = torch.device('cuda')
MO = MainOptuna('SAGE', device, loss, mode = 'unsupervised',number_of_trials=20)
MO.run(number_of_trials=10)

In [ ]:
device = torch.device('cuda')
MO = MainOptuna('GAT', device, loss, mode = 'unsupervised')
MO.run(number_of_trials=10)